### Words as data

We are expecting Michal Kosinski to join us either this session or the next. His work is at the center of the Cambridge Analytica story. He is now a faculty member at Stanford and, assuming all goes as planned, will give us a short overview of is work. We are taking the Cambridge Analytica story as an opportunity to think critically about algorithms. This means understanding something of how they work, the metaphors they draw on to create knowledge, the roles they play in society, and how we might interview them. 

While machine learning or AI or statistical modeling are becoming more important to how companies, communities, and countries operate, the area that Kosinski is involved in is particularly sensitive. The data they are working with can also be quite complex. We are going to start with the fact that they are analyzing status updates, social media posts, text. Text can be approached computationally in a number of ways. We are going to start simply and move out. 

Take a tweet. 

In [ ]:
%%HTML
<blockquote class="twitter-tweet" data-lang="en"><p lang="en" dir="ltr">Crazy Joe Biden is trying to act like a tough guy. Actually, he is weak, both mentally and physically, and yet he threatens me, for the second time, with physical assault. He doesn’t know me, but he would go down fast and hard, crying all the way. Don’t threaten people Joe!</p>&mdash; Donald J. Trump (@realDonaldTrump) <a href="https://twitter.com/realDonaldTrump/status/976765417908776963?ref_src=twsrc%5Etfw">March 22, 2018</a></blockquote>
<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

The simplest way to think of the sentence 

>Crazy Joe Biden is trying to act like a tough guy. Actually, he is weak, both mentally and physically, and yet he threatens me, for the second time, with physical assault. He doesn’t know me, but he would go down fast and hard, crying all the way. Don’t threaten people Joe!

is by breaking it into words and using those words as symbols. The presence or absence of a symbol might suggest something about the author or speaker. In 2014, Kosinski and his colleagues published a paper using this kind of lexical analysis to predict the age and gender of a person interacting with social media. Here is their paper and its abstract. It appeared in the *2014 Conference on Empirical Methods in Natural Language Processing (EMNLP)* and is titled [Developing Age and Gender Predictive Lexica over Social Media](http://www.aclweb.org/anthology/D14-1121).

>Demographic lexica have potential for
widespread use in social science, economic,
and business applications. We derive predictive
lexica (words and weights) for age and
gender using regression and classification
models from word usage in Facebook, blog,
and Twitter data with associated demographic
labels. The lexica, made publicly available,
achieved state-of-the-art accuracy in language
based age and gender prediction over Facebook
and Twitter, and were evaluated for
generalization across social media genres as
well as in limited message situations.

You can download the data associated with the paper [from an associated web site](http://mypersonality.org/wiki/doku.php?id=download_databases). Look for "Datasets Available Without Registration" and then "Language-based predictions." What you will get is gzipped file that unpacks into a folder with a couple of CSVs. Move them to the same folder as this notebook is located and open them with your favorite spreadsheet and have a look. Or use your new UNIX prowess!
<pre>
____|\
`-/    \
 (\_/)  \
 /_  _   |
 \/_/||) /
    '---' jg
</pre>

In [ ]:
%%sh
head emnlp14age.csv

You have two columns, one for `term`'s and one for `weight`'s. The paper describes how to use these weights. The data are better described [at this web site](http://www.wwbp.org/lexica.html) and at the bottom of the page you can "expand" to see how to apply the technique. Let's not wonder for the moment how these numbers were calculated, but instead see what kind of story they might tell.

First off, let's focus on `emnlp14age.csv` and tell me how you use these word-weight pairs? 

Now let's try it out. We are not going to use Pandas here but give you a little reminder about how to work with dictionaries and such. So let's open up the file and use a CSV `reader` object. We skim off the header...

In [ ]:
from csv import reader

data = reader(open("emnlp14age.csv"))
d = next(data)
print(d)

... and then read off the `_intercept`. It is thestart of the proces. The use of a sentence or document the estimate of someone's age is formed by simple additions to this number. 

In [ ]:
d = next(data)
print(d)

In [ ]:
intercept = float(d[1])

The easiest data structure for this exercise is a dictionary. We want numbers that are associated with words. So looking things up by word makes the most sense. We could torture Pandas into it, this is much cleaner. 

We will create a dictionary called `age` and then for each row in the data set, store the `weight` under the key `term`. 

In [ ]:
age = {}
for d in data:
    age[d[0]] = float(d[1])

In [ ]:
len(age)

In [ ]:
age.keys()

And let's have a look at some of these. 

In [ ]:
age["tough"]

In [ ]:
age["guy"]

Ideally we'd like a function that takes in some text and produces the estimate of the person's age. How should we do this? Take a second and write out the steps.

.

.

.

At the heart of any procedure will be simply checking to see if a word the text is contained in the dictionary `age` or not. Here we check for "crying."

In [ ]:
word = "crying"

if word in age:
    print(age[word])

In [ ]:
words = ["he","doesn't","know","me"]

score = intercept

for word in words:
    print(score)
    if word in age:
        score += age[word]/len(words)

print("Final age estimate:",score)

Here we create a function that will take the text and return the age score. We use TextBlob to parse out the words, but have to make an adjustment -- TextBlob makes "don't" into "don" and "'t" which is not how Kosinski and his colleagues coded things. So instead we will remove the single quotes, as "dont" and "cant" are there. To be completely above board, we'd have to deal with this, but it involves a little more work than it's worth right now.

Here's the function.

In [ ]:
def age_predict(text):
    
    # drop out single quotes
    text = sentence.replace("'","")
    
    # initialize the age estimate with the intercept
    score = 0
    n = 0

    # create a list of lowercase words to match the terms in our "age" dictionaray
    words = [str(w).lower() for w in TextBlob(text).words]

    # run through the words and create their contribution
    for word in words:
    
        if word in age:
            score += age[word]
            n += 1

    return(intercept + score/n)

Let's try this with Trump's tweet. Again, this algorithm gets better the more data it has to work with -- the more text of yours it sees. So one tweet won't be very accurate.

In [ ]:
from textblob import TextBlob

sentence = "Crazy Joe Biden is trying to act like a tough guy. Actually, he is weak, both mentally and physically, and yet he threatens me, for the second time, with physical assault. He doesn’t know me, but he would go down fast and hard, crying all the way. Don’t threaten people Joe!"

age_predict(sentence)

How about 2 tweets, then?

In [ ]:
sentence = "Crazy Joe Biden is trying to act like a tough guy. Actually, he is weak, both mentally and physically, and yet he threatens me, for the second time, with physical assault. He doesn’t know me, but he would go down fast and hard, crying all the way. Don’t threaten people Joe! Got $1.6 Billion to start Wall on Southern Border, rest will be forthcoming. Most importantly, got $700 Billion to rebuild our Military, $716 Billion next year...most ever. Had to waste money on Dem giveaways in order to take care of military pay increase and new equipment."
age_predict(sentence)

Here we can pull the last 20 tweets from the President.

In [ ]:
from tweepy import OAuthHandler, API
# setup the authentication

CONSUMER_KEY = "Urq5NyCqyjxiGF2gLoXg7o3UZ"
CONSUMER_SECRET = "KKiNtI8403O6R7MXUowWfM2mGB71eLJX2jeIMsgjGQ5SJrMaDl"
ACCESS_TOKEN = "20743-PbvM6FZjT2LoDSKTfAUpWwSwLKwPrXj25VVyIe5s3mya"
ACCESS_TOKEN_SECRET = "FdqcOey0FdwIhFhTyIuCJOFXwjFOX1EIDHG5vojPq3W51"

auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

# create an object we will use to communicate with the Twitter API
api = API(auth)

tweets = api.user_timeline("realdonaldtrump")

sentences = ""

for tweet in tweets:
    sentences = sentences + " "+tweet.text
    
print(sentences)

We should drop out things we know aren't in the `age` dictionary but are instead specific to Twitter -- basically drop the RT's and all the URLs.

In [ ]:
from re import sub

sentences = sub(r"RT","",sentences)
sentences = sub(r"(http[^ ]+)","",sentences)
print(sentences)

And what do we get?

In [ ]:
age_predict(sentences)

We could try more data, but the authors acknowledge that social media skews young, much younger than our president. It might be hard for this algorithm to reach his age in part because the training data doesn't really include folks like Trump. Tweets are pretty ragged also, typically, so this might not be the best test. 

That said, notice the way we are working here. The presence or absence of a word is used as a weight in a model. Positive weights tend to lift the estimate higher, lower weights push the estimate of age down. We could breakdown and bring the CSV into pandas and see what the ranges look like...

In [ ]:
agedf = read_csv("emnlp14age.csv")
agedf.head()

Let's have a look! Ah, plotly.

In [ ]:
from plotly.plotly import iplot, sign_in
import plotly.graph_objs as go

sign_in("cocteautt","8YLww0QuMPVQ46meAMaq")

myplot_parts = [go.Histogram(x=agedf["weight"])]
mylayout = go.Layout(autosize=False, width=800,height=400,margin=go.Margin(l=150,r=50,b=100,t=100,pad=4))
myfigure = go.Figure(data = myplot_parts, layout = mylayout)
iplot(myfigure,filename="age")

Which leads us to wonder about the really high and really low values... or even the values in betweeen. Let's sort the data and have a look. Keep in mind that some words just "go together" and this kind singleton analysis will miss those correlations. But still...

In [ ]:
agedf.sort_values("weight",inplace=True)
agedf.head(25)

In [ ]:
agedf.tail(25)

Does that make sense? What I'm after here is the narrative content in the algorithm. Having access to its internals, we can demystify and start to talk about it more sensibly -- even when it is as easy as this lexical weighting.

Read in the data set that assesses gender and tell me what you find. This style of reasoning, is also at the heart of many simple "sentiment analysis" packages. This group from Vermont looked at [happiness on Twitter](http://www.uvm.edu/~cdanfort/research/2011-hedonometer-arxiv.pdf) using a word-based approach. [Their lexicon and an expanded timeline of Twitter happiness is given here](https://hedonometer.org). 
<br><br>

<img src=https://github.com/computationaljournalism/columbia2018/raw/master/images/hap.jpg style="width: 75%; border: #000000 1px outset;"/>
<br>

Try it out!

### A geometry lesson

We are now going to switch to the paper at the center of the Cambridge Analytica story. The paper is called [Private traits and attributes are predictable from digital records of human behavior](http://www.pnas.org/content/110/15/5802) by Michal Kosinski, David Stillwell and Thore Graepel and appeared in the Proceedings of the National Academy of Science in April of 2013.

>myPersonality3 is a sample of personality scores and Facebook profile data that has been used in recent years for several different researches (i.e. Bachrach et al. 2012, Kosinski et al. 2013). It has been collected by David Stillwell and Michal Kosinski by means of a Facebook application that implements the Big5 test (Costa & McCrae’s NEO-PI-R domains and facets), among other psychological tests. The application obtained the consent from its users to record their data and use it for the research purposes. The dataset used for this workshop is a subset (250 users and about 9900 status updates) of the myPersonality sample. We selected only the users for which we had both information about personality and social network structure. The final dataset contains Facebook statuses in raw text, gold standard personality labels (self-assessments obtained using an 100-item long version of the IPIP personality questionnaire) and several social network measures, including: network size, betweenness centrality, density, brokerage and transitivity. We included personality labels both as scores and classes. Classes have been derived from scores with a median split, as for Essays. The status updates in myPersonality have been anonymized manually. For instance each proper name of person has been replaced with a fixed string (*PROPNAME*). Famous names, such as “Chopin” and “Mozart”, and locations, such as “New York” and “Mexico”, have not been replaced.

They have made their data available at the same site we [downloaded the gender and age data from.](http://mypersonality.org/wiki/doku.php?id=download_databases) Let's download it and read it in using Pandas and see what we have.

In [ ]:
from pandas import read_csv, set_option
set_option("display.max_colwidth",300)

person = read_csv("mypersonality_final.csv",encoding = "ISO-8859-1")
person.head(25)

People appear multiple times in this data set. We can see how many unique people we have and then get a sense of who posted a lot and who posted a little.

In [ ]:
person["#AUTHID"].value_counts().head()

In [ ]:
person["#AUTHID"].value_counts().shape

We are now going to shift gears and talk a bit about the kind of analysis that is at work here. For that we need to talk about the structure of data, specifically of tables.